In [1]:
%reload_ext autoreload
%autoreload 2

import torch
from torch.utils.data import DataLoader
from DatasetClass import KITTI_MOD_FIXED, ExtendedKittiMod
from torch.nn import Sigmoid
# from torchmetrics import IoU
from sklearn.metrics import jaccard_score

data_root = "/storage/remote/atcremers40/motion_seg/datasets/Extended_MOD_Masks/"
dataset = ExtendedKittiMod(data_root, test=False)
batch_size = 2
loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=False)


model_path = "/storage/remote/atcremers40/motion_seg/saved_models/27-12-2021_23-14_bs2/2_0.005_39.pt"
model = torch.load(model_path).to('cpu')
model.eval()

# needed to map to binary scores
sigmoid = Sigmoid()

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# iou = IoU(num_classes=2)

dirs loaded:
['2011_09_26_drive_0017_sync']


In [18]:
with torch.no_grad():
    model.eval()
    test_iou = []
    for idx, (data, targets) in enumerate(loader):
        data = data.float()
        targets = targets.int()

        # forward
        scores = model(data)
        scores_rounded = torch.round(sigmoid(scores))
        # targets = targets.detach().cpu()
        # scores_rounded = scores_rounded.detach().cpu()
        # iou = jaccard_score(scores_rounded.flatten().numpy(), targets.flatten().numpy())
        scores_rounded = scores_rounded.int()
        iou = iou_pytorch(scores_rounded, targets)
        test_iou.append(iou)
        if idx == 10:
            break

print(f"aIoU: {sum(test_iou)/len(test_iou)}")

aIoU: 0.4156779944896698
